# Q1


The stages are the weeks, from week 1 to week 12.

The states are the number of units carried over from the previous week.

The decision to be made is the number of units to attempt to repair.

The source of uncertainty is the probability that the repair operation for each unit will succeed or fail (85% chance of success and 15% chance of failure).

The value function f_t(i) represents the expected present value of the cost of the operation at week t with i units carried over from the previous week.



In [12]:
import numpy as np
from binomialPoisson import *

# Constants
num_weeks = 12
max_units = 15
repair_probs = binomial(8, 0.85)
repair_costs = np.array([0, 100, 110, 120, 130, 190, 200, 210, 220])
external_cost = 150
waiting_cost = 9
interest_rate = 0.001

# Initialize the value function table
value_table = np.zeros((num_weeks + 1, max_units + 1))
policy_table = np.zeros((num_weeks, max_units + 1))

# Iterate over weeks in reverse
for week in range(num_weeks - 1, -1, -1):
    # Iterate over the number of carried-over units
    for carried_units in range(max_units + 1):
        total_units = carried_units + 3
        min_cost = float('inf')
        best_repair_decision = -1

        # Iterate over possible repair decisions
        for repair_decision in range(min(8, total_units) + 1):
            expected_cost = repair_costs[repair_decision]
            waiting_units = total_units - repair_decision

            # Iterate over successful repairs
            for successful_repairs in range(repair_decision + 1):
                remaining_units = waiting_units - successful_repairs
                if remaining_units > max_units:
                    external_repair_units = remaining_units - max_units
                    remaining_units = max_units
                    expected_cost += external_cost * external_repair_units

                remaining_cost = waiting_cost * remaining_units + repair_probs[successful_repairs] * value_table[week + 1, remaining_units]
                expected_cost += remaining_cost / (1 + interest_rate)

            if expected_cost < min_cost:
                min_cost = expected_cost
                best_repair_decision = repair_decision

        value_table[week, carried_units] = min_cost
        policy_table[week, carried_units] = best_repair_decision

print("Optimal number of units to attempt to repair each week:")
print(policy_table)


Optimal number of units to attempt to repair each week:
[[0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 6. 7. 8. 8. 8. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Q2


Stages: The stages in the problem are the days t = 1, 2, ..., 10 before day 10.
States: The states within each stage are the number of available rooms for the night of day 10.
Decision: The decision to be made at each stage and state is the price to charge for the rooms on day 10.
Source of uncertainty: The source of uncertainty is the number of rooms that customers will attempt to book at a given price, modeled by a Poisson random variable with mean M(t, P).
Interpretation of the value function f_t(i): The value function f_t(i) represents the maximum expected revenue obtainable from the remaining i rooms on day t until day 10.

In [31]:
import math
import numpy as np
from binomialPoisson import *

# Python code for Poisson and Binomial distribution
#   Prof. Jonathan Eckstein

import numpy
import math


# Returns the distribution of a Poisson variable with mean "mean",
# truncated so its largest possible value is "maxVal".
#
# Uses that first term is exp(-M) and then
# Each successive term is previous one times M/i

def poisson(mean, maxVal):
    dist = np.zeros(maxVal+1)
    log_term = -mean
    for i in range(maxVal):
        log_dist_i = log_term + i * np.log(mean) - np.log(math.factorial(i))
        dist[i] = np.exp(log_dist_i)
    dist[maxVal] = np.exp(-np.sum(dist))
    return dist



# Returns the binomial distribution with n trials and p chance of
# success per trial
#
# Uses symmetry of the binomial coefficients (n choose k) and (n choose n-k)
# Only loops up through floor(n/2) and computes two terms per iteration
# Computes each binomial coefficient based on the previous one

def binomial(n,p) :
    dist = numpy.zeros(n+1)
    factorialTerm = 1.0
    k = 0
    while True :
        # Following two statements are redundant when k == n/2, but no matter
        dist[k]   = factorialTerm*(p**k)*(1-p)**(n-k)
        dist[n-k] = factorialTerm*(p**(n-k))*(1-p)**k
        factorialTerm *= (n - k)
        k += 1
        if k > n/2 :
            break
        factorialTerm /= k
    return dist

# Constants
beta0 = 3
beta1 = 0.01
beta2 = -0.01
beta3 = 0.0004
prices = [100, 150, 200, 250, 300]
days = 10
rooms = 20

# Mean function
def M(t, P):
    return (beta0 + beta1 * t) * math.exp((beta2 + beta3 * t) * P)

# Dynamic programming function
def hotel_yield_management():
    revenue = [[0] * (rooms + 1) for _ in range(days + 1)]

    for t in range(days - 1, -1, -1):
        for i in range(rooms + 1):
            max_rev = None
            for P in prices:
                mean = M(t + 1, P)
                max_val = min(i, math.ceil(mean))
                probs = poisson(max_val, math.ceil(mean))
                expected_rev = sum([probs[k] * (P * k + revenue[t + 1][i - k]) for k in range(max_val + 1)])
                if max_rev is None or expected_rev > max_rev:
                    max_rev = expected_rev
            revenue[t][i] = max_rev

    return revenue

# Calculate the dynamic programming solution and optimal pricing policy
revenue = hotel_yield_management()
policy = []

remaining_rooms = rooms

for t in range(days):
    max_rev = None
    best_price = 0
    for P in prices:
        max_val = min(remaining_rooms, math.ceil(M(t + 1, P)))
        probs = poisson(max_val, math.ceil(M(t + 1, P)))
        expected_rev = sum([probs[k] * (P * k + revenue[t + 1][remaining_rooms - k]) for k in range(max_val + 1)])
        if max_rev is None or expected_rev > max_rev:
            max_rev = expected_rev
            best_price = P
    policy.append(best_price)
    remaining_rooms -= best_price

# Output the results
print("Optimal pricing policy:", policy)
print("Maximum expected revenue:", revenue[0][20])

Optimal pricing policy: [300, 100, 100, 100, 100, 100, 100, 100, 100, 100]
Maximum expected revenue: 2969.326787165255


/var/folders/m6/wxc9fp790875bb7z5jw_q5c80000gn/T/ipykernel_3770/4032673825.py:22: RuntimeWarning: divide by zero encountered in log
  log_dist_i = log_term + i * np.log(mean) - np.log(math.factorial(i))
/var/folders/m6/wxc9fp790875bb7z5jw_q5c80000gn/T/ipykernel_3770/4032673825.py:22: RuntimeWarning: invalid value encountered in scalar multiply
  log_dist_i = log_term + i * np.log(mean) - np.log(math.factorial(i))
/var/folders/m6/wxc9fp790875bb7z5jw_q5c80000gn/T/ipykernel_3770/4032673825.py:22: RuntimeWarning: invalid value encountered in log
  log_dist_i = log_term + i * np.log(mean) - np.log(math.factorial(i))


In [33]:
pip install yasai

ERROR: Could not find a version that satisfies the requirement yasai (from versions: none)
ERROR: No matching distribution found for yasai
Note: you may need to restart the kernel to use updated packages.


In [35]:
import numpy as np
import pandas as pd

# Define input variables
num_techs = [2, 3, 4, 5]
phone_probs = [0.03, 0.04, 0.04, 0.07, 0.06, 0.07, 0.09, 0.05, 0.03, 0.04, 0.02, 0.03, 0.04, 0.06, 0.06, 0.07, 0.07, 0.05, 0.04, 0.03, 0.01]
major_prob = 0.28

# Define output variables
tech_hours = np.zeros(5)
total_hours = 0
phones_repaired = 0
total_cost = 0

# Define simulation model
def sim_model(num_techs):
    global tech_hours, total_hours, phones_repaired, total_cost
    tech_hours = np.zeros(5)
    total_hours = 0
    phones_repaired = 0
    total_cost = 0
    
    for day in range(1000):
        # Determine the number of phones to be repaired
        num_phones = np.random.choice(range(10, 31), p=phone_probs)
        
        # Determine the number of major repairs
        num_major = np.random.binomial(num_phones, major_prob)
        
        # Determine the number of minor repairs
        num_minor = num_phones - num_major
        
        # Assign phones to technicians
        phones_per_tech = num_minor // num_techs
        for i in range(num_techs):
            tech_hours[i] = phones_per_tech * 0.75 + (num_major * 0.75) / num_techs + (num_minor % num_techs) * 0.45 / num_techs
        
        # Calculate total hours and cost
        total_hours += tech_hours.sum()
        phones_repaired += num_phones
        if num_techs < num_minor:
            total_cost += num_minor * 0.45 + num_major * 0.75 + (num_minor - num_techs) * 0.85
        else:
            total_cost += num_minor * 0.45 + num_major * 0.75
        
    return total_cost / 1000

# Run simulations with different numbers of technicians
results = []
for n in num_techs:
    result = [sim_model(n) for i in range(100)]
    results.append(np.mean(result))

# Print the results
for i in range(len(num_techs)):
    print(f"Average cost per day with {num_techs[i]} technicians: {results[i]:.2f}")


Average cost per day with 2 technicians: 20.73
Average cost per day with 3 technicians: 19.86
Average cost per day with 4 technicians: 19.00
Average cost per day with 5 technicians: 18.13
